# Analisi e Implementazione di Metodi di Ottimizzazione

Creiamo i tre metodi di ottimizzazione per trovare i minimi di diverse funzioni:
1.  **Metodo del Gradiente (Gradient Descent - GD)**
2.  **Metodo di Newton**
3.  **Metodo del Gradiente Stocastico (SGD)**

Viene utilizzata una strategia di **Backtracking Line Search** per la determinazione del passo di apprendimento (alpha).

## 1 Import delle librerie

In [480]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
from mpl_toolkits.mplot3d import Axes3D # Per grafici 3D

## 2 Implementazione degli Algoritmi

## 2.1 Funzione Backtracking (Line Search)

Questa funzione implementa la ricerca del passo $\alpha$ tramite backtracking, basandosi sulla condizione di Armijo per garantire che il risultato converge alla soluzione.

**Parametri**:
- $f$: La funzione obiettivo
- $df$: Il gradiente della funzione obiettivo
- $X_k$: Il punto corrente (vettore)
- $p_k$: La direzione di discesa (vettore)

In [481]:
# Backtracking per alpha
def backtracking(f, df, X_k, p_k):
    alpha = 1
    rho = 1/2 # Fattore di riduzione per alpha
    c1 = 0.25

    # Calcola il gradiente del punto passato
    grad_k = df(X_k)

    # Calcola il prodotto scalare
    grad_dot_p = np.dot(grad_k, p_k)
    
    # Condizione Di Armijo: Il loop "while" continua FINCHÉ la condizione è VIOLATA (segno >)
    while f(X_k + alpha * p_k) > f(X_k) + c1 * alpha * grad_dot_p:
        alpha = rho * alpha

    return alpha


## 2.2 Metodo del gradiente

La direzione di discesa $p_k = -\nabla f(x)$, ovvero l'antigradiente

In [482]:
# Metodo del gradiente
def GD (f, df, X_old, maxit, tol_f, tol_x):
    
    count=0
    dim = len(X_old) # Calcolo la dimensione di X_old

    # Dichiare una matrice di 0 di dimensione maxit + 1 per dim
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0,] = X_old

    exit_flag = 'maxit' # Flag di uscita di default

    current_grad_norm=np.linalg.norm(df(X_old))

    # Controllo se ho superato le iterazioni o se ho raggiunto la tolleranza desiderata
    while count < maxit and current_grad_norm > tol_f:
        # Direzione di discesa
        p_k=-df(X_old)
        
        # Backtracking di alpha
        alpha=backtracking(f, df, X_old, p_k)

        # Calcolo di X_k+1
        X_new = X_old + alpha * p_k

        # Controllo se sto facendo progressi (tolleranza x)
        if np.linalg.norm(X_new-X_old) < tol_x:
            exit_flag = 'tol_x'
            break

        # Ricalcolo la norma del gradiente
        current_grad_norm=np.linalg.norm(df(X_new))

        # Aggiorno per l'iterazione successiva
        X_old = X_new
        count+=1

        # Aggiungo il valore della funzione in x_k nella matrice dei risultati 
        fun_history[count] = X_new

    # Controllo se l'uscita è dovuta a tol_f (norma del gradiente)
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    # Rimuovo le righe non utilizzate dalla cronologia
    if count<maxit:
        fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

## 2.2 Metodo Newton

La direzione di discesa è $p_k = - \frac{\nabla f_k}{H_k} $

Nota: questo metodo è molto lento perchè ad ogni iterazione si deve moltiplicare il gradiente e l'hessiana.

In [492]:
# Metodo di Newton
def Newton (f, df, hess_f, X_old, maxit, tol_f, tol_x):
    
    count=0
    dim = len(X_old) 
    
    # Cronologia
    fun_history = np.zeros((maxit + 1, dim))
    fun_history[0,]=X_old

    exit_flag = 'maxit'

    current_grad_norm=np.linalg.norm(df(X_old))

    while count < maxit and current_grad_norm > tol_f:

        # Calcolo l'Hessiana
        H_k = hess_f(X_old)

        # Calcolo il gradiente
        grad_k = df(X_old)

        # Direzione di discesa
        p_k=np.linalg.solve(H_k, -grad_k)

        # Backtracking di alpha
        alpha=backtracking(f, df, X_old, p_k)

        # Calcolo di X_k+1
        X_new = X_old + alpha * p_k

        # Controllo se sto facendo progressi (tolleranza x)
        if np.linalg.norm(X_new-X_old) < tol_x:
            exit_flag = 'tol_x'
            break

        # Ricalcolo la norma del gradiente
        current_grad_norm=np.linalg.norm(df(X_new))

        # Aggiorno per l'iterazione successiva
        X_old = X_new
        count+=1
        fun_history[count] = X_new

    
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    if count<maxit:
        fun_history = fun_history[:count+1]

    return X_old, count, fun_history, exit_flag

## 2.4 Metodo gradiente stocastico

In [409]:
# Metodo del gradiente stocastico
def SGD (df_completo, df_stocastico, X_old, n_samples, max_steps, tol_f, tol_x, k, alpha):

    # Calcolo la dimensione di X_old
    dim = len(X_old)

    # Inizializziamo il dataset S_k
    S_k = np.arange(n_samples)

    # Per la condizione di uscita
    step_count=0
    epoch_count=0

    # Dichiare una matrice di 0 di dimensione maxit+1 per dim
    fun_history = np.zeros((max_steps + 1, dim))
    fun_history[0,]=X_old

    # Caso che tutto va bene, esce per maxit
    exit_flag = 'maxit'

    current_grad_norm=np.linalg.norm(df_completo(X_old))

    # Controllo se ho superato le iterazioni o se ho raggiunto la tolleranza desiderata
    while step_count < max_steps and current_grad_norm > tol_f :

        # Randomizziamo gli indici del mini-batch
        np.random.shuffle(S_k)

        epoch_count += 1

        for i in range (0, n_samples, k):

            indices = S_k[i:i+k]

            # Selezioniamo il mini-batch
            # A_batch = A[indices, :]
            # b_batch = b[indices]
            
            # Calcola la direzione (gradiente stocastico)
            # grad_stoc = A_batch.T @ (A_batch @ X_old - b_batch)
            grad_stoc = df_stocastico(X_old, indices)
            p_k = -grad_stoc

            X_new = X_old + alpha * p_k

            # Controllo se sto facendo progressi (tolleranza x)
            if np.linalg.norm(X_new - X_old) < tol_x:
                exit_flag = 'tol_x'
                break

            # Aggiorno per l'iterazione successiva
            X_old = X_new
            step_count += 1
            fun_history[step_count] = X_new

            # Controllo se ho superato il numero di passi
            if step_count >= max_steps:
                break # Interrompe il 'for' loop

        # Ricalcolo la norma del gradiente "completo" per il check
        current_grad_norm=np.linalg.norm(df_completo(X_new))

        if exit_flag == 'tol_x':
            break

    # In caso siamo usciti per tolleranza
    if exit_flag == 'maxit' and current_grad_norm <= tol_f:
        exit_flag = 'tol_f'

    # Tronciamo l'array inizializzato all'inizio
    if step_count<max_steps: 
        fun_history = fun_history[:step_count+1]

    return X_old, step_count, fun_history, exit_flag, epoch_count

## 2.5 Funzione per la generazione dei grafici

In [487]:
def graph_generator (x0, y0, f,path_history_gd, path_history_newton, tmin, titolo):
    X_mesh, Y_mesh = np.meshgrid(x0, y0)
    Z = f([X_mesh, Y_mesh])

    # Memorizzo il percorso delle x e y
    path_x_gd = path_history_gd[:, 0]
    path_y_gd = path_history_gd[:, 1]
    path_x_new = path_history_newton[:, 0]
    path_y_new = path_history_newton[:, 1]

    # Definisco la figura
    fig = plt.figure(figsize=(14, 8))

    # <-- GENERAZIONE GRAFICO 3D -->

    # Aggiungo subplot 3D
    axs1 = fig.add_subplot(1,2,1,projection='3d')

    # Disegno la superficie
    axs1.plot_surface(X_mesh, Y_mesh, Z, cmap='viridis', rstride=1, cstride=1, linewidth=0, antialiased=True, alpha=0.7)

    # Aggiunge il percorso anche nel grafico 3D gradiente
    z_path_gd = f([path_x_gd, path_y_gd])
    axs1.plot(path_x_gd, path_y_gd, z_path_gd, 'r-o', label='Percorso del Gradiente')

    # Aggiunge il percorso anche nel grafico 3D newton
    z_path_newton = f([path_x_new, path_y_new])
    axs1.plot(path_x_new, path_y_new, z_path_newton, 'b-o', label='Percorso di Newton')

    # Evidenzia il punto iniziale e finale gradiente
    axs1.plot(path_x_gd[0], path_y_gd[0], 'go', label=f'Start: ({path_x_gd[0]}, {path_y_gd[0]})') # punto iniziale
    axs1.plot(path_x_gd[-1], path_y_gd[-1], 'rx', label=f'End: ({path_x_gd[-1]:.2f}, {path_y_gd[-1]:.2f})', markersize=16) # punto finale

    # Evidenzia il punto iniziale e finale newton
    axs1.plot(path_x_new[0], path_y_new[0], 'go', label=f'Start: ({path_x_new[0]}, {path_y_new[0]})') # punto iniziale
    axs1.plot(path_x_new[-1], path_y_new[-1], 'rx', label=f'End: ({path_x_new[-1]:.2f}, {path_y_new[-1]:.2f})', markersize=16) # punto finale

    axs1.plot(tmin[0], tmin[1], 'b*', markersize=15, label=f"Minimo Teorico ({tmin[0]}, {tmin[1]})") # Minimo teorico

    # Aggiungo label e titolo
    axs1.set_title(f"{titolo}")
    axs1.set_xlabel("x")
    axs1.set_ylabel("y")
    axs1.set_zlabel("z")
    axs1.legend()
    axs1.view_init(elev=30, azim=135) # Imposta un angolo di visuale

    # <--- GENERAZIONE GRAFICO 2D --->

    # Definisco il grafico 2D
    axs0 = fig.add_subplot(1,2,2)

    # Disegna le curve di livello
    c1 = axs0.contour(X_mesh, Y_mesh, Z, levels=50, cmap='viridis', alpha=0.7)
    fig.colorbar(c1, label='Valore di $f(x,y)$')

    # Disegna il percorso
    axs0.plot(path_x_gd, path_y_gd, 'r-o', label='Percorso del Gradiente')
    axs0.plot(path_x_new, path_y_new, 'b-o', label='Percorso di Newton')

    # Evidenzia il punto iniziale e finale gradiente
    axs0.plot(path_x_gd[0], path_y_gd[0], 'go', label=f'Start: ({path_x_gd[0]}, {path_y_gd[0]})') # punto iniziale
    axs0.plot(path_x_gd[-1], path_y_gd[-1], 'rx', label=f'End: ({path_x_gd[-1]:.2f}, {path_y_gd[-1]:.2f})', markersize=16) # punto finale
    
    # Evidenzia il punto iniziale e finale newton
    axs0.plot(path_x_new[0], path_y_new[0], 'go', label=f'Start: ({path_x_new[0]}, {path_y_new[0]})') # punto iniziale
    axs0.plot(path_x_new[-1], path_y_new[-1], 'bx', label=f'End: ({path_x_new[-1]:.2f}, {path_y_new[-1]:.2f})', markersize=16) # punto finale

    axs0.plot(tmin[0], tmin[1], 'y*', markersize=15, label=f"Minimo Teorico ({tmin[0]}, {tmin[1]})") # Minimo teorico
    # Etichette e titoli
    axs0.set_title(f"{titolo}")
    axs0.set_xlabel("x")
    axs0.set_ylabel("y")
    axs0.legend()
    axs0.set_aspect('equal', adjustable='box') # "Grafo è "quadrato"

    plt.tight_layout()
    plt.show()

### 2.6 Funzione per gli output

In [488]:
def get_solutions(f, df, h_f, x1, x2, maxit, tol_grad, tol_step):
    print("--- Metodo del Gradiente (GD) ---")
    sol_gd, iter_gd, path_gd, exit_gd = GD(f, df, x1, maxit, tol_grad, tol_step)
    print(f"Soluzione: {sol_gd}")
    print(f"Numero iterazioni: {iter_gd}")
    print(f"Condizione di uscita: {exit_gd}")

    print("--- Metodo di Newton ---")
    sol_newton, iter_newton, path_newton, exit_newton = Newton(f, df, h_f, x2, maxit, tol_grad, tol_step)
    print(f"Soluzione: {sol_newton}")
    print(f"Numero iterazioni: {iter_newton}")
    print(f"Condizione di uscita: {exit_newton}")

    return path_gd, path_newton

def get_SGD_solutions(f, sdf, x, n, maxit, tol_grad, tol_step, batch_k, learning_rate):
    print("--- Metodo del Gradiente Stocastico (SGD) ---")
    sol_sgd, iter_sgd, path_sgd, exit_sgd, epoch = SGD(f, sdf, x, n, maxit, tol_grad, tol_step, batch_k, learning_rate)
    print(f"Soluzione: {sol_sgd}")
    print(f"Numero iterazioni: {iter_sgd}")
    print(f"Condizione di uscita: {exit_sgd}")
    print(f"Epoche fatte: {epoch}")

    return path_sgd

--- 
## 3 Funzione Quadratica: $f(x, y) = (x-5)^2+(y-2)^2$

Cerchiamo i punti critici della funzione.

Calcoliamo la derivata rispetto a $x$ e a $y$.

- $\frac{∂x}{∂f} = ​2(x-5)$
- $\frac{∂y}{∂f} =​ 2(y-2)$

Ponendo $x=5$ e $y=2$, troviamo un punto critico.
Possiamo notare gli elementi della funzione sono elevati al quadrato, quindi sappiamo che il punto critico è un minimo globale.

**Nota:** Nei metodi utilizzati, converge in una sola iterazione.

### 3.1 Implementazione Dei Metodi

In [457]:
def f1(X):
    x = X[0]
    y = X[1]
    return (x-5)**2+(y-2)**2

def df1(X):
    x = X[0]
    y = X[1]

    df_dx = 2*(x-5) 
    df_dy = 2*(y-2)
    
    return np.array([df_dx, df_dy])

def hess_f1(X):
    HESS=np.zeros((2,2))

    HESS[0][0] = 2
    HESS[0][1] = 0
    HESS[1][0] = 0
    HESS[1][1] = 2
    return HESS    

x_range1=np.linspace(-2,12,200)
y_range1=np.linspace(-2,8,200)

maxit = 300
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto1_gd=np.array([0.0,0.0])
punto1_new=np.array([1.0,1.0])
tmin1 = [5.0, 2.0] # Minimo Teorico
titolo1 = r"$f(x, y) = (x-5)^2+(y-2)^2$"

### 3.2 Soluzioni dei Metodi

In [484]:
path_gd, path_new = get_solutions(f1, df1, hess_f1, punto1_gd, punto1_new, maxit, tolleranza_f, tolleranza_x)

--- Metodo del Gradiente (GD) ---
Soluzione: [5. 2.]
Numero iterazioni: 1
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [5. 2.]
Numero iterazioni: 1
Condizione di uscita: tol_f


### 3.2 Generazione Grafico

In [465]:
# Generazione grafico
graph_generator(x_range1, y_range1, f1, path_gd, path_new, tmin1, titolo1)

--- 
## 4. Funzione di Rosenbrock: $f(x, y) = (1-x)^2+100(y-x^2)^2$

Cerchiamo i punti critici della funzione.

Calcoliamo la derivata rispetto a $x$ e a $y$.

$\frac{df}{dx} = -2(1-x) + 200(y-x^2)(-2x) = 2(x-1) -400x(y-x^2) $

$\frac{df}{dy} =​ 200(y-x^2)$

Poniamo $y=x^2$, troviamo che $ 2(x-1) -400x(x^2-x^2)=0 \to 2x-2=0 \to x=1 $  

Quindi l'unico punto critico è su $(1,1)$

$H= \begin{bmatrix} 
\frac{d^2f}{dx^2} & \frac{d^2f}{dxdy} \\
\frac{d^2f}{dydx} & \frac{d^2f}{dy^2}
\end{bmatrix} =
\begin{bmatrix} 
 2-400y+1200x^2& -400x \\
-400x & 200
\end{bmatrix}$

Studiamo $\det(H)$.

$P(1,1) = 802\cdot 200 - 160000 = 160400 - 160000 = 400 > 0$, quindi è un punto di minimo

### 4.1 Implementazione dei Metodi

In [485]:
def f2(X):
    x = X[0]
    y = X[1]

    return (1-x)**2+100*(y-x**2)**2

def df2(X):
    x = X[0]
    y = X[1]

    df_dx = -2*(1-x) - 400*(y-x**2)*x
    df_dy = 200*(y-x**2)
    
    return np.array([df_dx, df_dy])

def hess_f2(X):
    HESS=np.zeros((2,2)) # Crea una matrice 2x2
    x = X[0]
    y = X[1]

    HESS[0][0] = 2 - 400*y + 1200*x**2 # Derivata seconda rispetto a x
    HESS[0][1] = -400*x # Derivata mista
    HESS[1][0] = HESS[0][1] # Per teorema di Sxhwarz
    HESS[1][1] = 200 # Derivata seconda rispetto a y
    return HESS
    

x_range2=np.linspace(-2,2,200)
y_range2=np.linspace(-1,3,200)

maxit = 5000
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto2_gd=np.array([0.0,0.0])
punto2_new=np.array([0.5,0.5])
tmin2 = [1.0,1.0]
titolo2 = r"$f(x, y) = (1-x)^2+100(y-x^2)^2$"

### 4.2 Soluzioni dei Metodi

In [486]:
path_gd_r, path_newton_r = get_solutions(f2, df2, hess_f2, punto2_gd, punto2_new, maxit, tolleranza_f, tolleranza_x)

--- Metodo del Gradiente (GD) ---
Soluzione: [0.9998038  0.99960895]
Numero iterazioni: 2200
Condizione di uscita: tol_x
--- Metodo di Newton ---
Soluzione: [0.99999971 0.99999935]
Numero iterazioni: 10
Condizione di uscita: tol_x


### 4.3 Generazione Grafico

In [453]:
# Grafico del metodo GD
graph_generator(x_range2, y_range2, f2, path_gd_r, path_newton_r, tmin2, titolo2)

---
## 5 Funzione multimodale: $f(x,y) = x^4-x^2+y^2$

Cerchiamo i punti critici della funzione.

Calcoliamo la derivata rispetto a $x$ e a $y$.

- $\frac{df}{dx} = ​4x^3-2x = 2x(2x^2-1) $
- $\frac{df}{dy} =​ 2y$

Notiamo che ponendo $x=0$ e $y= 0$ troviamo il nostro primo punto critico.

Ponendo $y=0$, troviamo i casi in cui $2x^2-1=0$, ovvero $x= \pm \sqrt{\frac12}$, ovvero $\simeq 0.70717$

Calcoliamo se questi punti critici sono punti minimi, massimi o di sella.

$H= \begin{bmatrix} 
\frac{d^2f}{dx^2} & \frac{d^2f}{dxdy} \\
\frac{d^2f}{dydx} & \frac{d^2f}{dy^2}
\end{bmatrix} =
\begin{bmatrix} 
12x^2-2 & 0 \\
0 & 2
\end{bmatrix}$

Studiamo $ \det(H)$.

$P(0,0) = -4 < 0$, quindi è un punto di sella

$P(\sqrt{\frac12},0) = 2(6-2) = 8 > 0$, quindi è un punto di minimo

$P(-\sqrt{\frac12},0) = 2(6-2) = 8 >0$, quindi è un punto di minimo

### 5.1 Implementazione dei Metodi

In [475]:
def f3(X):
    x = X[0]
    y = X[1]

    return x**4-x**2+y**2

def df3(X):
    x = X[0]
    y = X[1]

    df_dx = 4*x**3-2*x
    df_dy = 2*y
    
    return np.array([df_dx, df_dy])

def hess_f3(X):
    HESS=np.zeros((2,2))
    x = X[0]
    y = X[1]

    HESS[0][0] = 12*x**2-2 # Derivata seconda rispetto a x
    HESS[0][1] = 0 # Derivata mista
    HESS[1][0] = HESS[0][1] # Per teorema di Sxhwarz, la derivata mista è uguale
    HESS[1][1] = 2 # Derivata seconda rispetto a y
    return HESS
    

x_range3=np.linspace(-1.5,1.5,200)
y_range3=np.linspace(-1.5,1.5,200)

maxit = 5000
tolleranza_f = 1.e-6
tolleranza_x = 1.e-6
punto3_neg=np.array([-1.0,-1.0]) 
punto3_pos=np.array([1.0,1.0]) 
punto3_sad=np.array([0.0,5.0]) 
tmin3 = [0.0, 0.0]
tmin3_alt = [-np.sqrt(1/2), 0.0]
titolo3 = r"$f(x,y) = x^4-x^2+y^2$"

### 5.2 Soluzioni dei Metodi

In [476]:
path_gd_m, path_new_m = get_solutions(f3, df3, hess_f3, punto3_pos, punto3_neg, maxit, tolleranza_f, tolleranza_x)

--- Metodo del Gradiente (GD) ---
Soluzione: [0. 0.]
Numero iterazioni: 1
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [-0.70710712  0.        ]
Numero iterazioni: 4
Condizione di uscita: tol_x


### 5.3 Generazione Grafico

In [477]:
# Gradiente
graph_generator(x_range3, y_range3, f3, path_gd_m, path_new_m, tmin3, titolo3)

/home/dule/anaconda3/envs/CN25/lib/python3.13/tkinter/__init__.py:862: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


---
## 6. Generatore grafici errore

D'ora in poi lavoreremo in $\R^n$, quindi non possiamo visualizzarli con dei grafici, ma possiamo avere dei grafici per gli errori

In [511]:
def error_graph(f, df, gd, newton, sgd, titolo):
    errori_gd = [f(x_k) for x_k in gd]
    errori_newton = [f(x_k) for x_k in newton]
    errori_sgd = [f(x_k) for x_k in sgd]

    norma_grad_gd = [np.linalg.norm(df(x_k)) for x_k in gd]
    norma_grad_newton = [np.linalg.norm(df(x_k)) for x_k in newton]
    norma_grad_sgd = [np.linalg.norm(df(x_k)) for x_k in sgd]

    # Crea il grafico di GD e Newton
    fig, axs = plt.subplots(2, 2, figsize=(16, 10)) # Dimensione più bilanciata per 2x2

    # --- Grafico [0, 0]: Errore (f) per GD e Newton ---
    ax = axs[0, 0]
    ax.plot(errori_gd, label="Gradient Descent (GD)", marker='.', linestyle='-')
    ax.plot(errori_newton, label="Newton Smorzato", marker='.', linestyle='--')
    ax.set_title("Valore Funzione (GD vs Newton)")
    ax.set_ylabel("Valore Funzione $f(x_k)$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)

    # --- Grafico [0, 1]: Errore (f) per SGD ---
    ax = axs[0, 1]
    ax.plot(errori_sgd, label="Stochastic GD", color='green', alpha=0.8)
    ax.set_title("Valore Funzione (SGD)")
    ax.set_ylabel("Valore Funzione $f(x_k)$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)

    # --- Grafico [1, 0]: Norma Gradiente per GD e Newton ---
    ax = axs[1, 0]
    ax.plot(norma_grad_gd, label="Gradient Descent (GD)", marker='.', linestyle='-')
    ax.plot(norma_grad_newton, label="Newton Smorzato", marker='.', linestyle='--')
    ax.set_title("Norma Gradiente (GD vs Newton)")
    # Usiamo una stringa raw (r"...") per il LaTeX
    ax.set_ylabel(r"Norma Gradiente $\|\nabla f(x_k)\|$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)

    # --- Grafico [1, 1]: Norma Gradiente per SGD ---
    ax = axs[1, 1]
    ax.plot(norma_grad_sgd, label="Stochastic GD", color='green', alpha=0.8)
    ax.set_title("Norma Gradiente (SGD)")
    ax.set_ylabel(r"Norma Gradiente $\|\nabla f(x_k)\|$ (log scale)")
    ax.set_xlabel("Iterazione (k)")
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", linestyle='--', linewidth=0.5)

    # --- 3. Titolo finale e Layout ---
    fig.suptitle(f"Analisi Convergenza per {titolo}", fontsize=18)
    
    # Applica tight_layout UNA SOLA VOLTA, con l'aggiustamento per il suptitle
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    plt.show()

---
# 7. Minimi Quadrati Lineari: $f(x) = \frac12||Ax-b||_2^2$ con $A \in \R^{n \times n}$

Definiamo la $i$-esima funzione componente come: $f_i(x) = \frac{1}{2} ( A_ix-b_i )^2$

Per la linearità dell'operatore gradiente, il gradiente della somma è la somma dei gradienti:
$\nabla f(x) = \nabla \left( \sum_{i=1}^m f_i(x) \right) = \sum_{i=1}^m \nabla f_i(x)$

Applichiamo la regola della catena:
$\nabla f_i(x) = \frac{1}{2} \cdot 2 \cdot (A_i x - b_i) \cdot \nabla(A_i x - b_i)$

Ora, calcoliamo $\nabla(A_i x - b_i)$. La funzione $g(x) = A_i x - b_i$ è:
$g(x) = (a_{i1}x_1 + a_{i2}x_2 + \dots + a_{in}x_n) - b_i$

Calcoliamo le derivate parziali rispetto a ogni $x_j$:
$\frac{\partial g}{\partial x_j} = \frac{\partial}{\partial x_j} (a_{i1}x_1 + \dots + a_{ij}x_j + \dots + a_{in}x_n - b_i) = a_{ij}$

Assemblando queste derivate parziali in un vettore gradiente (colonna):
$$ \nabla(A_i x - b_i) = 
   \begin{bmatrix} 
   \frac{\partial g}{\partial x_1} \\ \vdots \\ \frac{\partial g}{\partial x_n} 
   \end{bmatrix} = 
   \begin{bmatrix} 
   a_{i1} \\ \vdots \\ a_{in} 
   \end{bmatrix} = A_i^T $$
(Questo è il trasposto della *riga* $A_i$).

Sostituendo questo risultato, otteniamo il gradiente della $i$-esima componente:
$\nabla f_i(x) = (A_i x - b_i) A_i^T$
Questo è un vettore $n \times 1$ (uno scalare moltiplicato per un vettore colonna).

**Ora sommiamo i gradienti di tutte le $m$ componenti:**

$\nabla f(x) = \sum_{i=1}^m \nabla f_i(x) = \sum_{i=1}^m (A_i x - b_i) A_i^T$

Definiamo il vettore "errore" $e = Ax - b$, dove $e_i = (A_i x - b_i)$ è la $i$-esima componente (scalare). La somma diventa:
$\nabla f(x) = \sum_{i=1}^m e_i A_i^T$

Analizziamo la matrice $A^T \in \R^{n \times m}$. Le **colonne** di $A^T$ sono i vettori $A_i^T$:
$$ A^T = 
   \begin{bmatrix} 
   | & | & & | \\ 
   A_1^T & A_2^T & \dots & A_m^T \\ 
   | & | & & | 
   \end{bmatrix} $$

La moltiplicazione matrice-vettore $A^T e$ è definita come la combinazione lineare delle colonne di $A^T$ con i pesi (scalari) $e_i$:
$$ A^T e = 
   \begin{bmatrix} 
   | & \dots & | \\ 
   A_1^T & \dots & A_m^T \\ 
   | & \dots & | 
   \end{bmatrix} 
   \begin{bmatrix} 
   e_1 \\ e_2 \\ \vdots \\ e_m 
   \end{bmatrix} 
   = e_1 A_1^T + e_2 A_2^T + \dots + e_m A_m^T = \sum_{i=1}^m e_i A_i^T $$

Quindi $\nabla f(x) = A^T (Ax - b)$

I punti minimi sono $A^T(Ax-b)=0$

Prendendo il gradiente ottenuto, notiamo che l'Hessiana è semplicemente $A^TA$, ovvero una costante.

Questo significa che la curvatura del grafico è uguale ad ogni punto.

**Nota: questa è una loss function:**

- $A$ sono i dati, ovvero il dataset in input

- $x$ sono i parametri del modello, ovvero i $ \theta$

- $b$ sono i valori reali

- $Ax-b$ sono le previsioni del modello - i valori reali: sono la distanza tra i valori predetti e quelli reali

Lo eleviamo al quadrato per avere una distanza positiva e al quadrato per dare importanza ai valori distanti a quelli predetti.

Quindi, questa funzione serve per quantificare quanto è "sbagliato" il nostro modello.

## 7.1 Implementazione dei Metodi

In [512]:
# Definisco la funzione
def f4(x, A, b):
    e = A @ x -b
    norm_sq = np.dot(e, e)
    return 1/2 * norm_sq

# Questa calcola il GRADIENTE COMPLETO (Batch)
def df4(X, A, b):
    return A.T @ (A @ X - b)

# Hessiana
def hess_f4(A):
    return A.T @ A

# Questa calcola il GRADIENTE STOCASTICO (Mini-Batch)
def sdf4(x, indices, A, b):
    A_batch= A[indices, :]
    b_batch= b[indices]
    
    return A_batch.T @ (A_batch @ x - b_batch)

n = 50 # siamo in R^50
A = np.random.rand(n,n) # A è una matrice n x n con valori casuali
b = A @ np.ones(n) # rendiamo b in modo che b= A(vettori 1)+rumore
X_start_1 = np.zeros(n) # Punto di partenza
X_start_2 = np.random.randn(n) # Punto di partenza

title_f4 = r"$f(x) = \frac{1}{2}\|Ax-b\|_2^2 \text{ con } A \in \mathbb{R}^{n \times n}$"
maxit = 2000
maxit_SGD = 20000          # L'SGD richiede molti più passi del GD
tol_grad = 1e-3            # Tolleranza più alta per SGD
tol_step = 1e-6
batch_k = 5                # Dimensione del mini-batch
learning_rate = 0.001      # Alpha (deve essere molto piccolo)

f_min_sq = lambda x: f4(x, A, b) # Equivalente a scrivere def f_min_sq(x): return f4(x, A, b)
df_min_sq = lambda x: df4(x, A, b) 
h_min_sq = lambda x: hess_f4(A)
sdf_min_sq = lambda x, idx: sdf4(x, idx, A, b)

## 7.2 Soluzioni

In [513]:
path_gd, path_newton = get_solutions(f_min_sq, df_min_sq, h_min_sq, X_start_1, X_start_2, maxit, tol_grad, tol_step)

path_sgd = get_SGD_solutions(f_min_sq, sdf_min_sq, X_start_1, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)

--- Metodo del Gradiente (GD) ---
Soluzione: [0.97519061 1.01093011 0.98679271 0.99965344 0.99117447 0.9796362
 1.02753856 0.99356038 0.9990253  1.01128273 0.95323749 0.992687
 1.00370235 1.01182285 1.00595534 0.99676261 1.04531556 1.01145568
 1.01441443 1.01868763 0.99424453 0.99961065 0.99819499 0.98806095
 1.01616353 1.00208672 1.01518164 1.02263773 0.97722314 1.00353077
 0.99175659 1.00717957 0.98288752 1.01896003 0.99333814 1.03581297
 1.00988266 0.97743614 0.99124382 0.97057299 1.02172699 0.97846902
 1.00803449 0.98048603 0.99762589 0.98999394 0.99781803 0.98703508
 0.96462229 1.02449622]
Numero iterazioni: 2000
Condizione di uscita: maxit
--- Metodo di Newton ---
Soluzione: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Numero iterazioni: 1
Condizione di uscita: tol_f
--- Metodo del Gradiente Stocastico (SGD) ---
Soluzione: [0.97859318 0.98787286 0.99209988 0.99116388 0.985

## 7.3 Generazione Grafico

In [515]:
error_graph(f_min_sq, df_min_sq, path_gd, path_newton, path_sgd, title_f4)

Notiamo che newton è velocissimo. Questo perchè in funzioni quadrate, salta direttamente in quel punto.
Il grafico del gradiente è rumoroso, perchè nella iterazione k fa un passo fuori rotta, il grafico sale, per poi essere ritornato nella iterazione k+1. 

---
# 8 Minimi quadrati con regolarizzazione: $f(x) = \frac{1}{2}||Ax-b||^2_2 + \lambda ||x||^2_2$, con $\lambda \in [0,1]$

Questa funzione viene chiamata anche come Regressione di Ridge.

Ora dobbiamo trovare il termine noto $\frac{1}{2}||Ax-b||^2_2$ e il termine di regolarizzazione $\lambda ||x||^2_2$

La medesima, vuole trovare una x con componenti (pesi) il più possibile vicini a zero. Penalizza soluzioni con valori molto grandi.

Separiamo il problema come $\nabla f(x) = \nabla f_{LS}(x) + \nabla f_{Reg}(x)$

Troviamo il gradiente:

- Come già fatto nella funzione precedente, $\nabla f_{LS}(x) = A^T (Ax - b)$

- $\nabla f_{Reg} = 2 \lambda x $

Quindi la soluzione è $\nabla f(x) = A^T (Ax - b) + 2\lambda x$

Per l'Hessiana, scomponiamo la funzione $H=f_A+f_\lambda$. 

$f_A$ l'abbiamo già definita nel caso di funzione precedente $A^TA$

$f_\lambda$, invece, ha le derivate per $H_{ki}$ con $k != i$ sono uguali a $0$ perchè sono costanti.

Mentre per $k=i$, ovvero la matrice diagonale, non vengono considerate costanti e la loro derivata è $2\lambda$, 

Quindi $A^T A + 2\lambda I$, dove I è la matrice identità.

## 8.1 Definisco le funzioni

In [425]:
# Definisco la funzione
def f5(x, A, b, lambda_val):
    e = A @ x -b
    errore_ls = 1/2 * np.dot(e, e)
    errore_reg = lambda_val*np.dot(x,x)
    return errore_ls + errore_reg

# Questa calcola il GRADIENTE COMPLETO (Batch)
def df5(x, A, b, lambda_val):
    grad_ls = A.T @ (A @ x - b)
    grad_reg = 2*lambda_val*x
    return grad_ls +  grad_reg

# Hessiana
def hess_f5(A, lambda_val):
    return A.T @ A + 2 * lambda_val * np.identity(n)

# Questa calcola il GRADIENTE STOCASTICO (Mini-Batch)
def sdf5(x, indices, A, b, lambda_val):
    A_batch= A[indices, :]
    b_batch= b[indices]

    grad_ls_stoc = A_batch.T @ (A_batch @ x - b_batch)
    grad_reg = 2 * lambda_val * x
    
    return grad_ls_stoc + grad_reg

n = 50 # siamo in R^50
A = np.random.rand(n,n) # A è una matrice n x n con valori casuali
X_true = np.ones(n)
b = A @ X_true + 0.1 * np.random.randn(n) # rendiamo b in modo che b= A(vettori 1)+rumore
lambda_val = 0.1 

titolo_f5 = r"$f(x) = \frac{1}{2}\|Ax-b\|_2^2 + \lambda \|x\|_2^2 \text{ con } \lambda \in [0,1]$"
X_start = np.zeros(n)      # Punto di partenza
maxit = 2000
maxit_SGD = 20000         # L'SGD richiede molti più passi del GD
batch_k = 5                # Dimensione del mini-batch
learning_rate = 0.001      # Alpha (deve essere molto piccolo)
# NOTA: Una epoca è fatta ad ogni 10 iterazioni, perchè n/batch_k = 50 / 5 = 10

f_min_sqr = lambda x: f5(x, A, b, lambda_val)
df_min_sqr = lambda x: df5(x, A, b, lambda_val)
h_min_sqr = lambda x: hess_f5(A, lambda_val)
sdf_min_sqr = lambda x, idx: sdf5(x, idx, A, b, lambda_val)

## 8.2 Soluzioni

In [426]:
path_gd, path_newton = get_solutions(f_min_sqr, df_min_sqr, h_min_sqr, X_start, maxit, tol_grad, tol_step)

path_sgd = get_SGD_solutions(df_min_sqr, sdf_min_sqr, X_start, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)


--- Metodo del Gradiente (GD) ---
Soluzione: [1.06603086 0.97743522 1.07151049 0.91687968 1.00009479 1.00161952
 0.96194733 1.07698235 1.06220052 0.96751674 0.98199613 0.98576459
 0.9939642  0.97934077 0.99020506 1.03806357 0.95034622 1.02480236
 1.04694916 1.08182595 1.00392421 0.91045892 1.07331167 1.03845134
 1.15889071 0.97477527 1.08017719 1.01951742 1.06709794 0.8711588
 1.00482695 0.97680848 0.97332964 0.99154753 0.8991189  1.08265919
 1.16534341 0.89961414 1.02529299 0.77224158 1.03105206 0.98023906
 0.91462646 0.96034272 1.04439518 0.9201893  0.94399081 1.02223214
 0.92964854 0.99556451]
Numero iterazioni: 2000
Condizione di uscita: maxit
--- Metodo di Newton ---
Soluzione: [1.06321086 0.97630728 1.07449091 0.91534675 0.99929624 1.00108313
 0.95993377 1.07649205 1.06307519 0.96776841 0.98040756 0.98538491
 0.99294281 0.98212466 0.99428892 1.03731166 0.95059729 1.02434415
 1.05021492 1.08517905 1.00486834 0.90934919 1.07763386 1.03972994
 1.15919729 0.97320576 1.08041085 1.0184

## 8.3 Generazione Grafico

In [427]:
error_graph(f_min_sqr, path_gd, path_newton, path_sgd, titolo_f5)

---
# 9 $f(x) = \sum \limits_{i=1}^n (x_i-i)^2-\sum \limits_{i=1}^n \ln(x_i)$, con $x_i>0$

$\nabla f(x) = 2 (x-i) - \frac{1}{x}$

Notiamo che per $ i!= k$,$H_{ik}$ non ha elementi perchè la derivata rispetto a x_k ha solo costanti.

I punti minimi sono: $2 (x-i) - \frac{1}{x} = 0$ 

Ovvero $2x^2-2ix-1 = 0 \to \frac{2i\pm\sqrt{4i^2+8}}{4}$

Invece, per $i = k$, $H_{ik} = 2+(\frac{1}{x^2})$

Quindi l'Hessiana ha solo elementi nella diagonale

**Nota**: qua il metodo stocastico è inutile, dato che il gradiente non ha matrici e il metodo del gradiente ha già un costo $O(n)$.

In [428]:
def f6 (x, idx):
    if np.any(x <= 0):
        return np.inf # ritorna infinito se x non è nel dominio (ln(x) con x<=0)

    sum1 = np.sum((x-idx)**2)
    sum2 = np.sum(np.log(x))
    return sum1 - sum2
    
def df6 (x, idx):
    return 2 * (x - idx) - 1/x

def hess_f6 (x):
    HESS = 2+(1/(x**2))
    return np.diag(HESS)

def sdf6(x, mini_batch, n, idx):

    x_batch = x[mini_batch]
    idx_batch = idx[mini_batch]
    grad_stoc = np.zeros(n)
    
    # Calcola il gradiente solo per quei componenti
    grad_batch_components = 2 * (x_batch - idx_batch) - (1 / x_batch)
    
    # Inserisci i componenti calcolati nel vettore zero
    grad_stoc[mini_batch] = grad_batch_components
    
    return grad_stoc

title_f6 = r"$f(x) = \sum_{i=1}^n (x_i-i)^2 - \sum_{i=1}^n \ln(x_i)$"
idx = np.arange(1, n+1)
X_start = np.ones(n)
b = A @ X_start + 0.1 * np.random.randn(n)
n = 50

f_lambda = lambda x: f6(x, idx)
df_lambda = lambda x: df6(x, idx)
h_lambda = lambda x: hess_f6(x)
sdf_lambda = lambda x, k: sdf6(x, k, n, idx)

In [429]:
path_gd, path_newton = get_solutions(f_lambda, df_lambda, h_lambda, X_start, maxit, tol_grad, tol_step)

path_sgd = get_SGD_solutions(df_lambda, sdf_lambda, X_start, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)


--- Metodo del Gradiente (GD) ---
Soluzione: [ 1.36585366  2.22474227  3.1583123   4.12132034  5.09807621  6.082207
  7.07071421  8.0620192   9.05521679 10.04975247 11.04526825 12.04152299
 13.03834842 14.03562364 15.03325959 16.0311892  17.02936105 18.02773504
 19.02627944 20.02496883 21.02378259 22.02270384 23.02171862 24.02081528
 25.01998403 26.01921657 27.01850583 28.01784577 29.01723114 30.01665742
 31.01612065 32.01561738 33.01514456 34.01469953 35.01427989 36.01388353
 37.01350858 38.01315334 39.0128163  40.0124961  41.0121915  42.01190139
 43.01162476 44.0113607  45.01110837 46.010867   47.01063589 48.01041441
 49.01020196 50.009998  ]
Numero iterazioni: 6
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [ 1.3660254   2.22474461  3.15831201  4.12132005  5.09807602  6.08220688
  7.07071414  8.06201915  9.05521676 10.04975245 11.04526824 12.04152298
 13.03834841 14.03562363 15.03325958 16.0311892  17.02936105 18.02773504
 19.02627944 20.02496883 21.02378259 22.022

In [430]:
error_graph(f_lambda, path_gd, path_newton, path_sgd, title_f6)

---
# 10 $f(x) = \sum \limits_{i=1}^n (x_i-b_i)^2+c\sum \limits_{i=1}^n \ln(x_i)$

$\nabla f(x) = 2(x_i-b_i) + c\frac{1}{x_i}$

I punti minimi sono $2(x_i-b_i) + c\frac{1}{x_i} = 0 \to 2x_i^2-2x_ib_i+c = 0 \to \frac{2b_i\pm\sqrt{4b_i^2-8c}}{4}$

**Nota**: dato che abbiamo $\ln(x_i)$, quando la funzione ha $x\simeq 0$, si trova $-\infty$. Quindi non esiste un minimo locale.

Scomponiamo la funzione in $f(x) = f_{xb} + f_{ln}$

Per l'Hessiana, se deriviamo rispetto a $x_k$:

$f_{xb} = 2$, ovvero una $2\cdot I$, la matrice identità

$f_{ln} = -c\frac{1}{x^2}$, sempre nella matrice diagonale.

Quindi $H_{ik} = 2-c\frac{1}{x^2}$

**Note**: questa Hessiana non è sempre definita positiva, quindi è più difficile minimizzare.

- Il metodo di Newton convergerà verso il risultato, il metodo del gradiente non funziona e quello del SGD converge molto lentamente.

In [431]:
def f7(x,c,b):

    sum1 = np.sum((x-b)**2)
    sum2 = c*np.sum(np.log(x))

    return sum1+sum2

def df7(x,c,b):
    return 2*(x-b)+c*(1/x)

def hess_f7(x,c):
    HESS =2-c*(1/(x**2))
    return np.diag(HESS)

def sdf7(x, c, b, n, mini_batch):
    x_batch = x[mini_batch]
    b_batch = b[mini_batch]
    grad_stoc = np.zeros(n)
    
    # Calcola il gradiente solo per quei componenti
    grad_batch_components = 2 * (x_batch - b_batch) + c*(1 / x_batch)
    
    # Inserisci i componenti calcolati nel vettore zero
    grad_stoc[mini_batch] = grad_batch_components
    
    return grad_stoc

title_f7 = r"$f(x) = \sum_{i=1}^n (x_i-b_i)^2+c\sum_{i=1}^n \ln(x_i)$"
b = np.arange(1, n+1)
X_start = np.ones(n) # Partiamo da tutte x=1
c= -2.0 # DEVE essere negativo

f_c = lambda x: f7(x, c, b)
df_c = lambda x: df7(x, c, b)
h_c = lambda x: hess_f7(x, c)
sdf_c = lambda x, k: sdf7(x, c, b, n, k)

In [432]:
path_gd, path_newton = get_solutions(f_c, df_c, h_c, X_start, maxit, tol_grad, tol_step)

path_sgd = get_SGD_solutions(df_c, sdf_c, X_start, n, maxit_SGD, tol_grad, tol_step, batch_k, learning_rate)


--- Metodo del Gradiente (GD) ---
Soluzione: [ 1.61818182  2.41421393  3.30277564  4.23606798  5.1925824   6.16227766
  7.14005494  8.12310563  9.10977223 10.09901951 11.09016994 12.08276253
 13.07647322 14.07106781 15.06637298 16.06225775 17.05862138 18.05538514
 19.05248659 20.04987562 21.04751155 22.04536102 23.04339638 24.04159458
 25.0399362  26.03840481 27.03698637 28.03566885 29.03444185 30.03329638
 31.03222457 32.03121954 33.03027525 34.02938637 35.02854814 36.02775638
 37.02700731 38.02629759 39.02562419 40.02498439 41.02437575 42.02379604
 43.02324325 44.02271555 45.02221126 46.02172887 47.02126697 48.0208243
 49.02039967 50.01999201]
Numero iterazioni: 9
Condizione di uscita: tol_f
--- Metodo di Newton ---
Soluzione: [ 1.61803279  2.41420118  3.30275694  4.2360515   5.19257028  6.16226925
  7.14004919  8.12310166  9.10976946 10.09901755 11.09016853 12.08276149
 13.07647245 14.07106723 15.06637253 16.0622574  17.05862111 18.05538492
 19.05248641 20.04987548 21.04751144 22.04

In [433]:
error_graph(f_c, path_gd, path_newton, path_sgd, title_f7)

invalid command name "127261938136000process_stream_events"
    while executing
"127261938136000process_stream_events"
    ("after" script)
